In [1]:
from datetime import datetime
import requests
import numpy as np
import pandas as pd
import yaml
import re
from bs4 import BeautifulSoup
import wikitextparser as wtp
from ratelimit import rate_limited
from operator import *

def save_obj(obj, name ):
    import pickle
    with open('obj/'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)
        
def load_obj(name ):
    import pickle
    with open('obj/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

teamEasy เปลี่ยนชื่อทีมจาก prizepool เป็น teamName

In [21]:
def parseSquadandPrize(tourdict, teamEasy):
    '''
    input: title of tournament
    output: dictionary of participants of that tournament
    '''
    json = tourdict['wikitext']
    wiki = json.get('parse').get('wikitext').get('*')
    wiki = re.sub('\n', '', wiki)
    parsed = wtp.parse(wiki)
    templates = parsed.templates
    memt = {}
    memp = {}
    squad = {}
    listteamprize = []
    for t in templates:
        if str(t)[:12] == '{{prize pool':
            for ar in t.arguments:
                if (str(ar)[:6] == '|place'):
                    place = ar.value
                if (str(ar)[:9] == '|usdprize'):
                    usdprize = ar.value           
                if str(ar)[1:] == ar.value:
                    teamshort = re.sub(r' ', '', ar.value)
                    try:
                        teamEz = teamEasy[teamshort]
                    except KeyError:
                        teamEz = teamshort
                    teamEz = re.sub(r'\W+|_', '', teamEz).lower()
                    try:
                        prize[teamEz] = {}
                        listteamprize.append(teamEz)
                        squad[teamEz]['usdprize'] = usdprize
                        squad[teamEz]['place'] = place
                    except KeyError:
                        continue 
        if str(t)[:11] == '{{TeamCard|':
            for ar in t.arguments:
                if (str(ar)[:5] == '|team'):
                    teamname = re.sub(r'\W+', '', ar.value).lower()
                    here = False
                    try:
                        squad[teamname]
                    except KeyError:
                        for teamprize in listteamprize:
                            if (teamname in teamprize) or (teamprize in teamname):
                                squad[teamname] = squad[teamprize]
                                here = True
                                break
                        if not here:
                            print(teamname)
                            print(listteamprize)
                            ip = input()
                            if ip == 'break':
                                break
                            squad[teamname] = squad[ip]
                            memt[teamname] = ip
                            memp[ip] = teamname
                            here = False
                for i in range(1, 6):
                    if (str(ar)[:4] == '|p' + str(i) + '='): 
                        pid = re.sub(r'\W+', '', ar.value).lower()
                        try:
                            pid = AlternateIdDict[re.sub(r'\W+', '', ar.value).lower()]
                        except KeyError:
                            pid = re.sub(r'\W+', '', ar.value).lower()
                        squad[teamname]['pos' + str(i)] = re.sub(r'\W+', '', pid).lower()
                    if (str(ar)[:7] == '|p' + str(i) + 'link'):
                        squad[teamname]['pos' + str(i)] = squad[teamname]['pos' + str(i)] + ' haslink'                                          
    return squad

In [244]:
tourdict = load_obj('forelo/' + re.sub(r'\W+', '', '2013 WPC ACE Dota 2 League'.lower() ))

In [25]:
json = tourdict['wikitext']
wiki = json.get('parse').get('wikitext').get('*')
wiki = re.sub('\n', '', wiki)
parsed = wtp.parse(wiki)
templates = parsed.templates
squad = {}
listteamprize = []
for t in templates:
    if str(t)[:12] == '{{prize pool':
        for ar in t.arguments:
            if (str(ar)[:6] == '|place'):
                place = ar.value
            if (str(ar)[:9] == '|usdprize'):
                usdprize = ar.value           
            if str(ar)[1:] == ar.value:
                teamshort = re.sub(r' ', '', ar.value)
                try:
                    teamEz = teamEasy[teamshort]
                except KeyError:
                    teamEz = teamshort
                teamEz = re.sub(r'\W+|_', '', teamEz).lower()
                try:
                    squad[teamEz] = {}
                    listteamprize.append(teamEz)
                    squad[teamEz]['usdprize'] = usdprize
                    squad[teamEz]['place'] = place
                except KeyError:
                    continue 
    if str(t)[:11] == '{{TeamCard|':
        for ar in t.arguments:
            if (str(ar)[:5] == '|team'):
                teamname = re.sub(r'\W+', '', ar.value).lower()
                here = False
                try:
                    squad[teamname]
                except KeyError:
                    for teamprize in listteamprize:
                        if (teamname in teamprize) or (teamprize in teamname):
                            squad[teamname] = squad[teamprize]
                            here = True
                            break
                    if not here:
                        print(teamname)
                        print(listteamprize)
                        ip = input()
                        if ip == 'break':
                            break
                        squad[teamname] = squad[ip]
                        here = False
            for i in range(1, 6):
                if (str(ar)[:4] == '|p' + str(i) + '='): 
                    pid = re.sub(r'\W+', '', ar.value).lower()
                    try:
                        pid = AlternateIdDict[re.sub(r'\W+', '', ar.value).lower()]
                    except KeyError:
                        pid = re.sub(r'\W+', '', ar.value).lower()
                    squad[teamname]['pos' + str(i)] = re.sub(r'\W+', '', pid).lower()
                if (str(ar)[:7] == '|p' + str(i) + 'link'):
                    squad[teamname]['pos' + str(i)] = squad[teamname]['pos' + str(i)] + ' haslink'                    

thegdbteam
['natusvincere', 'teamempire', 'gdteam', 'pulseesports', 'moscowfive', 'meetyourmakers', 'virtuspro']
gdteam
absolutelegends
['natusvincere', 'teamempire', 'gdteam', 'pulseesports', 'moscowfive', 'meetyourmakers', 'virtuspro']
break


In [28]:
for lqn, pdc in squad.items():
    try:
        teamName = comparedict[lqn]
    except KeyError:
        teamName = lqn
    j = 0
    try:
        place = pdc.pop('place')
        prize = pdc.pop('usdprize')
    except KeyError:
        break
    for srt in startelodict:
        for ert in endelodict:
            if (teamName == re.sub(r'\W+', '', srt['teamName']).lower()) & (teamName == re.sub(r'\W+', '', ert['teamName']).lower()):
                j += 1
                for pos, id in pdc.items():
                    df = df.append({
                        'team': teamName,
                        'pos': pos,
                        'id': id,
                        'tour': ifd['title'],
                        'prizepool': ifd['prize'],
                        'place': place,
                        'prize': prize,
                        'start': ifd['start'],
                        'end': ifd['end'],
                        'startelo': srt['elo64']['current'],
                        'startglicko2mu': srt['glicko2']['mu'],
                        'startglicko2phi': srt['glicko2']['phi'],
                        'startglicko2rating': srt['glicko2']['rating'],
                        'endelo': ert['elo64']['current'],
                        'endglicko2mu': ert['glicko2']['mu'],
                        'endglicko2phi': ert['glicko2']['phi'],
                        'endglicko2rating': ert['glicko2']['rating']
                    }, ignore_index=True)
    if j == 0:
        for pos, id in pdc.items():
            if id == '':
                continue
            else:
                df = df.append({
                    'team': teamName,
                    'pos': pos,
                    'id': id,
                    'tour': ifd['title'],
                    'prizepool': ifd['prize'],
                    'place': place,
                    'prize': prize,                        
                    'start': ifd['start'],
                    'end': ifd['end'],
                    'startelo': np.nan,
                    'startglicko2mu': np.nan,
                    'startglicko2phi': np.nan,
                    'startglicko2rating': np.nan,
                    'endelo': np.nan,
                    'endglicko2mu': np.nan,
                    'endglicko2phi': np.nan,
                    'endglicko2rating': np.nan
                }, ignore_index=True)

In [26]:
df

,team,pos,id,tour,prizepool,place,prize,start,end,startelo,startglicko2mu,startglicko2phi,startglicko2rating,endelo,endglicko2mu,endglicko2phi,endglicko2rating
0,teamempire,pos1,blowyourbrain,RaidCall Dota 2 League Season 1,10000,1,"5,000",10/09/2012,18/11/2012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,teamempire,pos2,scandal,RaidCall Dota 2 League Season 1,10000,1,"5,000",10/09/2012,18/11/2012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,teamempire,pos3,funn1k,RaidCall Dota 2 League Season 1,10000,1,"5,000",10/09/2012,18/11/2012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,teamempire,pos4,goblak,RaidCall Dota 2 League Season 1,10000,1,"5,000",10/09/2012,18/11/2012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,teamempire,pos5,silent,RaidCall Dota 2 League Season 1,10000,1,"5,000",10/09/2012,18/11/2012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,evilgeniuses,pos1,fear,RaidCall Dota 2 League Season 1,10000,2,"3,000",10/09/2012,18/11/2012,1127.178191,1739.804399,58.251412,1594.175869,1102.30554,1757.679651,45.318269,1644.383978
6,evilgeniuses,pos2,jeyo,RaidCall Dota 2 League Season 1,10000,2,"3,000",10/09/2012,18/11/2012,1127.178191,1739.804399,58.251412,1594.175869,1102.30554,1757.679651,45.318269,1644.383978
7,evilgeniuses,pos3,demon,RaidCall Dota 2 League Season 1,10000,2,"3,000",10/09/2012,18/11/2012,1127.178191,1739.804399,58.251412,1594.175869,1102.30554,1757.679651,45.318269,1644.383978
8,evilgeniuses,pos4,maelk,RaidCall Dota 2 League Season 1,10000,2,"3,000",10/09/2012,18/11/2012,1127.178191,1739.804399,58.251412,1594.175869,1102.30554,1757.679651,45.318269,1644.383978
9,evilgeniuses,pos5,bdiz,RaidCall Dota 2 League Season 1,10000,2,"3,000",10/09/2012,18/11/2012,1127.178191,1739.804399,58.251412,1594.175869,1102.30554,1757.679651,45.318269,1644.383978


In [3]:
def getEloDict(tourdict):
    return (tourdict['startelodict'], tourdict['endelodict'])

In [32]:
def compareAndPut(df, squad, ifd, startelodict, endelodict):
    for lqn, pdc in squad.items():
        try:
            teamName = comparedict[lqn]
        except KeyError:
            teamName = lqn
        j = 0
        try:
            place = pdc.pop('place')
            prize = pdc.pop('usdprize')
        except KeyError:
            break
        for srt in startelodict:
            for ert in endelodict:
                if (teamName == re.sub(r'\W+', '', srt['teamName']).lower()) & (teamName == re.sub(r'\W+', '', ert['teamName']).lower()):
                    j += 1
                    for pos, id in pdc.items():
                        df = df.append({
                            'team': teamName,
                            'pos': pos,
                            'id': id,
                            'tour': ifd['title'],
                            'prizepool': ifd['prize'],
                            'place': place,
                            'prize': prize,
                            'start': ifd['start'],
                            'end': ifd['end'],
                            'startelo': srt['elo64']['current'],
                            'startglicko2mu': srt['glicko2']['mu'],
                            'startglicko2phi': srt['glicko2']['phi'],
                            'startglicko2rating': srt['glicko2']['rating'],
                            'endelo': ert['elo64']['current'],
                            'endglicko2mu': ert['glicko2']['mu'],
                            'endglicko2phi': ert['glicko2']['phi'],
                            'endglicko2rating': ert['glicko2']['rating']
                        }, ignore_index=True)
        if j == 0:
            for pos, id in pdc.items():
                if id == '':
                    continue
                else:
                    df = df.append({
                        'team': teamName,
                        'pos': pos,
                        'id': id,
                        'tour': ifd['title'],
                        'prizepool': ifd['prize'],
                        'place': place,
                        'prize': prize,                        
                        'start': ifd['start'],
                        'end': ifd['end'],
                        'startelo': np.nan,
                        'startglicko2mu': np.nan,
                        'startglicko2phi': np.nan,
                        'startglicko2rating': np.nan,
                        'endelo': np.nan,
                        'endglicko2mu': np.nan,
                        'endglicko2phi': np.nan,
                        'endglicko2rating': np.nan
                    }, ignore_index=True)
        
    return df

In [5]:
alltour2 = {}

In [7]:
alltour2['2013 WPC ACE Dota 2 League'] = alltour['2013 WPC ACE Dota 2 League']

In [8]:
alltour2

{'2013 WPC ACE Dota 2 League': {'end': '01/01/2014',
  'prize': 247920,
  'start': '22/09/2013',
  'title': '2013 WPC ACE Dota 2 League'}}

In [29]:
alltour = load_obj('/dict/touranddate')
comparedict = load_obj('/dict/lqtodat')
AlternateIdDict = load_obj('/dict/AlternateIdDict')
teamEasy = load_obj('/dict/teamEasy2')

In [30]:
endti2 = '2012-09-02'
endti7 = '2017-08-12'

In [33]:
%%time
errortitle = []
df = pd.DataFrame(columns=['team', 'pos', 'id', 'tour', 'prizepool', 'place', 'prize', 'start', 'end',
                           'startelo', 'startglicko2mu', 'startglicko2phi', 'startglicko2rating', 'endelo', 'endglicko2mu', 'endglicko2phi', 'endglicko2rating'])
for turl, ifd in alltour.items():
    try:
        if (pd.to_datetime(ifd['start'], format='%d/%m/%Y') > pd.to_datetime(endti2)) & (pd.to_datetime(ifd['end'], format='%d/%m/%Y') <= pd.to_datetime(endti7)):
            easyTourTitle = re.sub(r'\W+', '', ifd['title']).lower()
            tourdict = load_obj('forelo/' + easyTourTitle )
            ptc = parseSquadandPrize(tourdict, teamEasy)
            startelodict, endelodict = getEloDict(tourdict)
            print(ifd)
            df = compareAndPut(df, ptc, ifd, startelodict, endelodict)
    except:
        print(turl)
        errortitle = errortitle.append(turl)

{'title': 'WCG Asian Championship 2012', 'start': '08/09/2012', 'end': '09/09/2012', 'prize': 6000}
{'title': 'RaidCall Dota 2 League Season 1', 'start': '10/09/2012', 'end': '18/11/2012', 'prize': 10000}
thegdbteam
['natusvincere', 'teamempire', 'gdteam', 'pulseesports', 'moscowfive', 'meetyourmakers', 'virtuspro']
gdteam
absolutelegends
['natusvincere', 'teamempire', 'gdteam', 'pulseesports', 'moscowfive', 'meetyourmakers', 'virtuspro']
break
{'title': 'StarLadder StarSeries Season 3', 'start': '17/09/2012', 'end': '21/10/2012', 'prize': 15000}
{'title': 'GosuLeague Season 4', 'start': '02/10/2012', 'end': '28/10/2012', 'prize': 'Nan'}
{'title': 'GosuLeague Season 4 Division 1', 'start': '02/10/2012', 'end': '28/10/2012', 'prize': 5000}
{'title': 'The Premier League Season 3', 'start': '03/10/2012', 'end': '13/01/2013', 'prize': 6000}
{'title': 'G-1 Champions League Season 4', 'start': '22/10/2012', 'end': '18/11/2012', 'prize': 52879.2}
{'title': 'Electronic Sports World Cup 2012', 

break
keitagaming
['mousesports', 'virtuspro', 'fnatic']
break
nextkz
['mousesports', 'virtuspro', 'fnatic']
break
uebelstgamyng
['mousesports', 'virtuspro', 'fnatic']
break
roxkis
['mousesports', 'virtuspro', 'fnatic']
break
tcmgaming
['mousesports', 'virtuspro', 'fnatic']
break
teamdignitas
['mousesports', 'virtuspro', 'fnatic']
break
teamempire
['mousesports', 'virtuspro', 'fnatic']
break
{'title': 'Corsair Gaming Summer Tournament 2013', 'start': '28/06/2013', 'end': '26/07/2013', 'prize': 10000}
{'title': 'Nexon Starter League', 'start': '06/07/2013', 'end': '14/07/2013', 'prize': 19760}
myonmyon
['teamzenith', 'orangeesports', 'mith', 'joenet']
break
isgvindicate
['teamzenith', 'orangeesports', 'mith', 'joenet']
break
ipod2
['teamzenith', 'orangeesports', 'mith', 'joenet']
break
netolicsg
['teamzenith', 'orangeesports', 'mith', 'joenet']
break
insidiousidol
['teamzenith', 'orangeesports', 'mith', 'joenet']
break
firstdeparture
['teamzenith', 'orangeesports', 'mith', 'joenet']
bre

AttributeError: 'NoneType' object has no attribute 'append'

In [36]:
df.sort_values('prize')

,team,pos,id,tour,prizepool,place,prize,start,end,startelo,startglicko2mu,startglicko2phi,startglicko2rating,endelo,endglicko2mu,endglicko2phi,endglicko2rating
357,absolutelegends,pos3,freezer,StarLadder StarSeries Season 4,15000,5-16,0,13/11/2012,23/12/2012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1060,qpadredpandas,pos2,singsing,WePlay Dota2 League Season 1,8700,7-8,0,09/03/2013,13/05/2013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1059,qpadredpandas,pos1,wagamama,WePlay Dota2 League Season 1,8700,7-8,0,09/03/2013,13/05/2013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1058,mousesports,pos5,alex,WePlay Dota2 League Season 1,8700,5-6,0,09/03/2013,13/05/2013,1088.913635,1673.455100,48.038805,1553.358087,1130.947610,1679.882583,37.898595,1585.136095
1057,mousesports,pos4,pas,WePlay Dota2 League Season 1,8700,5-6,0,09/03/2013,13/05/2013,1088.913635,1673.455100,48.038805,1553.358087,1130.947610,1679.882583,37.898595,1585.136095
1056,mousesports,pos3,qojqva,WePlay Dota2 League Season 1,8700,5-6,0,09/03/2013,13/05/2013,1088.913635,1673.455100,48.038805,1553.358087,1130.947610,1679.882583,37.898595,1585.136095
1055,mousesports,pos2,fata,WePlay Dota2 League Season 1,8700,5-6,0,09/03/2013,13/05/2013,1088.913635,1673.455100,48.038805,1553.358087,1130.947610,1679.882583,37.898595,1585.136095
1054,mousesports,pos1,black,WePlay Dota2 League Season 1,8700,5-6,0,09/03/2013,13/05/2013,1088.913635,1673.455100,48.038805,1553.358087,1130.947610,1679.882583,37.898595,1585.136095
1061,qpadredpandas,pos3,select,WePlay Dota2 League Season 1,8700,7-8,0,09/03/2013,13/05/2013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1053,teamdignitas,pos5,waytosexy,WePlay Dota2 League Season 1,8700,5-6,0,09/03/2013,13/05/2013,1061.865382,1717.899106,45.645411,1603.785579,1183.352534,1740.941735,36.220413,1650.390701


In [253]:
#Clean Data
df = df[df['id'] != '']
df.loc[(df['id'] == 'g haslink'), 'id'] = 'g'
df.loc[(df['id'] == 'noone haslink'), 'id'] = 'noone'
df.loc[(df['id'].str[:8] == 'paparazi'), 'id'] = 'paparazi'
df.pos = pd.to_numeric(df.pos.str[-1])

In [9]:
#df[['team', 'pos', 'id', 'tour', 'prizeusd', 'start', 'startelo', 'end', 'endelo']].to_csv('elo.csv', encoding='utf-8', index=False)
#df[['team', 'pos', 'id', 'tour', 'prizeusd', 'start', 'startglicko2mu', 'startglicko2phi', 'startglicko2rating', 'end', 'endglicko2mu', 'endglicko2phi', 'endglicko2rating']].to_csv('glicko.csv', encoding='utf-8', index=False)

In [110]:
%%time
errortitle = []
for turl, ifd in alltour.items():
    try:
        if (pd.to_datetime(ifd['start'], format='%d/%m/%Y') > pd.to_datetime(endti2)) & (pd.to_datetime(ifd['end'], format='%d/%m/%Y') <= pd.to_datetime(endti7)):
            easyTourTitle = re.sub(r'\W+', '', ifd['title']).lower()
            tourdict = load_obj('forelo/' + easyTourTitle )
            json = tourdict['wikitext']
            wiki = json.get('parse').get('wikitext').get('*')
            wiki = re.sub('\n', '', wiki)
            parsed = wtp.parse(wiki)
            templates = parsed.templates
            squad = {}
            listteamcard = []
            listteamprize = []            
            for t in templates:
                if str(t)[:11] == '{{TeamCard|':
                    for ar in t.arguments:
                        if (str(ar)[:5] == '|team'):
                            teamname = re.sub(r'\W+', '', ar.value).lower()
                            listteamcard.append(teamname)
                if str(t)[:12] == '{{prize pool':
                    for ar in t.arguments:
                        if (str(ar)[:6] == '|place'):
                            place = ar.value
                        if (str(ar)[:9] == '|usdprize'):
                            usdprize = ar.value           
                        if str(ar)[1:] == ar.value:
                            teamshort = re.sub(r' ', '', ar.value)
                            try:
                                teamEz = teamEasy[teamshort]
                            except KeyError:
                                teamEz = teamshort
                            listteamprize.append(re.sub(r'\W+|_', '', teamEz).lower())
        has = set(listteamprize).intersection(listteamcard)
        set(listteamprize).difference(has)
        print(easyTourTitle)
        print('team that match         : ' + str(has))
        print('listteamprize not match : ' + str(set(listteamprize).difference(has)))
        print('listteamcard not match  : ' + str(set(listteamcard).difference(has)))
        print('-' * 10)

    except:
        print(turl)
        errortitle = errortitle.append(turl)

theinternational2017specialevents
team that match         : set()
listteamprize not match : {'teamradiant', 'teamdire'}
listteamcard not match  : set()
----------
theinternational2017specialevents
team that match         : set()
listteamprize not match : {'teamradiant', 'teamdire'}
listteamcard not match  : set()
----------
theinternational2017specialevents
team that match         : set()
listteamprize not match : {'teamradiant', 'teamdire'}
listteamcard not match  : set()
----------
theinternational2017specialevents
team that match         : set()
listteamprize not match : {'teamradiant', 'teamdire'}
listteamcard not match  : set()
----------
theinternational2017specialevents
team that match         : set()
listteamprize not match : {'teamradiant', 'teamdire'}
listteamcard not match  : set()
----------
theinternational2017specialevents
team that match         : set()
listteamprize not match : {'teamradiant', 'teamdire'}
listteamcard not match  : set()
----------
theinternational2017sp

thepremierleagueseason4
team that match         : {'absolutelegends', 'virtuspro', 'teamempire', 'teamliquid', 'alliance', 'mousesports', 'kaipi', 'teamdignitas', 'rox'}
listteamprize not match : {'fnatic'}
listteamcard not match  : {'fnaticeu'}
----------
starladderstarseriesseason5
team that match         : {'absolutelegends', 'virtuspro', 'teamempire', 'powerrangers', 'notidehunter', 'natusvincere', '4friendschrillee', '3dclan', 'dddota', 'tcmgaming', 'rox', 'freedomoftheaction', 'mousesports', '3dmax', 'lionspride'}
listteamprize not match : {'fnatic'}
listteamcard not match  : {'fnaticeu'}
----------
armaggeddondota2grandslamasia2013
team that match         : {'awake', 'applebananacucumber'}
listteamprize not match : {'departure'}
listteamcard not match  : {'harrosh', 'gizmogaming', 'genuinenessofgeeks', 'gameware', 'firstdeparture', 'thekade', 'neolutionindonesia', 'sobliquegamings', 'pacificesports', 'srinbins', 'tlerreborn', 'gangsterblaque'}
----------
weplaydota2leagueseason1

nexonsponsorshipleagueseason1
team that match         : {'ave', 'moonlightunited', 'aceonline', 'mvphot6ix', 'startale', 'forourutopia', 'eyesoftiger', 'seasonone', 'fastgg', 'underrateds', 'sushiandyasha', 'virtualthrone'}
listteamprize not match : {'symphony', 'njkids', 'victorylight', 'placebum'}
listteamcard not match  : {'svictorylights', 'splacebums', 'symphonydota', 'teamnjkids'}
----------
dota2championsleagueseason1
team that match         : {'virtuspro', 'speedgamingint', 'natusvincere', 'fnatic', 'sigmaint', 'teamliquid', 'alliance', 'mousesports', 'teamdignitas', 'evilgeniuses'}
listteamprize not match : set()
listteamcard not match  : set()
----------
mlgfallinvitational
team that match         : {'typicalmistakes', 'nom', 'teamdignitas'}
listteamprize not match : {'joikernaught'}
listteamcard not match  : {'thejoikernaught'}
----------
fragbitemasters2013
team that match         : {'speedgamingint', 'fiskestanga', 'youboat', 'qpadredpandas', 'ursastroithramm', 'warmaker',

hyperxd2lwesternchallenge
team that match         : {'teamempire', 'natusvincere', 'fnatic', 'cloud9', 'teamliquid', 'alliance', 'evilgeniuses', 'rox'}
listteamprize not match : set()
listteamcard not match  : set()
----------
fragbitemasters2014
team that match         : {'meetyourmakers', 'teamempire', 'powerrangers', 'fnatic', 'trickedesport', 'nextkz', 'sigmaint', 'teamliquid', 'vastcast', 'turtlemasters', '4friendschrillee', 'teamdog', 'rox'}
listteamprize not match : {'mmc'}
listteamcard not match  : {'monomaniac'}
----------
techlabscup2014season1
team that match         : {'virtuspro', 'teamempire', 'nextkz', 'rox'}
listteamprize not match : set()
listteamcard not match  : set()
----------
americandotaleagueseason2
team that match         : {'isurusgaming', 'osirisgaming', 'uniongaming', 'top5', 'sneakynyxassassins', 'revengeesports', 'shadowsofthepast', 'teamliquid', 'mousesports', 'teamehug'}
listteamprize not match : {'noearth', 'natusvincerenorthamerica', 'cnb'}
listteamcar

worldcyberarena2014
team that match         : {'vicigaming', 'lgdgaming', 'natusvincere', 'cloud9', 'teammalaysia', 'alliance', 'invictusgaming', 'newbee'}
listteamprize not match : set()
listteamcard not match  : set()
----------
asusrogdreamleagueseason2
team that match         : {'virtuspro', 'mvpphoenix', 'teamtinker', 'virtuspropolar', 'cloud9', 'alliance', 'evilgeniuses'}
listteamprize not match : {'4asc'}
listteamcard not match  : {'lgdgaming', 'teamempire', 'sneakynyxassassins', 'fnaticeu', 'complexitygaming', 'invictusgaming', '4anchorsseacaptain', 'xgamekz', 'rave'}
----------
brasilgamecup2014
team that match         : {'thehouseisdown', 'uniongaming', 'keydstars', 'paingaming', 'nottoday'}
listteamprize not match : {'intz', 'isurus', 'cnb'}
listteamcard not match  : {'isurusgaming', 'intzesports', 'cnbesportsclub'}
----------
eslonenewyork2014
team that match         : {'vicigaming', 'natusvincere', 'sneakynyxassassins', 'cloud9', 'alliance', 'evilgeniuses', 'teamsecret'}
l

joindotamlgproleagueseason2
team that match         : {'virtuspro', 'ninjasinpyjamas', 'fnatic', 'summersrift', 'alliance', 'mousesports', 'londonconspiracy', 'evilgeniuses'}
listteamprize not match : set()
listteamcard not match  : set()
----------
rogplayitcooldota2invitational
team that match         : {'virtuspro', 'alliance', 'ninjasinpyjamas', 'lemondogs'}
listteamprize not match : set()
listteamcard not match  : set()
----------
worldcyberarena2015americanproqualifiers
team that match         : {'voidboys', 'teamleviathan', 'nottoday', 'northamericanrejects', 'rootgaming', 'summersrift', 'teamehug', 'wheelwhreckwhilewhistling'}
listteamprize not match : set()
listteamcard not match  : set()
----------
marstvdota2league2015springchineseleague
team that match         : {'hypergloryteam', 'ehome', 'tongfu', 'cdecgaming', 'immortalmagnetogaming', 'newbee', 'teamdk'}
listteamprize not match : {'lgdgaming'}
listteamcard not match  : {'lgd'}
----------
mpglseason7sea
team that match   

worldcyberarena2016chinesequalifierss1
team that match         : {'cdecyouth', 'wingsgaming', 'tongfu', 'vicigamingreborn', 'invictusgaming', 'newbee'}
listteamprize not match : {'forthedream', 'ib'}
listteamcard not match  : {'insanebuster', 'ftdcluba'}
----------
russianesportscup2016
team that match         : set()
listteamprize not match : {'teamslark', '4kaverage', 'cocacoliki', 'dott', 'nyanpasu', 'chewtoy', 'alphasense', 'blessedbygod', 'ehshasbi', '3d', 'sqreenssquad', 'flipsid3tactics', 'cascade', 'dvk', 'политехник'}
listteamcard not match  : set()
----------
starladderileagueinvitationalseason1
team that match         : {'virtuspro', 'lgdgaming', 'mvpphoenix', 'og', 'natusvincere', 'vegasquadron', 'vicigamingreborn', 'alliance'}
listteamprize not match : set()
listteamcard not match  : set()
----------
eslonemanila2016
team that match         : {'wingsgaming', 'teamempire', 'ehome', 'fnatic', 'teamliquid', 'complexitygaming', 'teamsecret'}
listteamprize not match : {'mineski

worldelectronicsportsgames2016asiapacificfinals
team that match         : {'teamnation', 'mvpphoenix', 's1lykos', 'tobeornottobe', 'iclen', 'slayers', 'tncproteam', 'nextgeneration', 'dateletubbies', 'signaturetrust', 'nolifer5reborn', '90sgaming', 'powergaming', 'mvpaegis', 'maxy', 'detonator'}
listteamprize not match : set()
listteamcard not match  : set()
----------
worldcyberarena2016amqualifiers
team that match         : {'uniongamingbo', 'vinceteipsum', 'pichau', 'nottoday', 'resnonverba', 'midasclubelite', 'apologaming', 'alphapro'}
listteamprize not match : set()
listteamcard not match  : set()
----------
thesummit6
team that match         : {'wingsgaming', 'virtuspro', 'og', 'ehome', 'teamnp', 'digitalchaos', 'teamfaceless', 'evilgeniuses'}
listteamprize not match : set()
listteamcard not match  : set()
----------
asusrogdreamleagueseason6
team that match         : {'alliance', 'theimperial', 'teamliquid', 'escapegaming'}
listteamprize not match : set()
listteamcard not match 

In [109]:
re.sub(r'\W+|_', '', 'eg._K').lower()

'egk'